In [1]:
!pip install autogluon

ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.6, <3.9; 0.1.0b20210207 Requires-Python >=3.6, <3.8; 0.1.0b20210208 Requires-Python >=3.6, <3.8; 0.1.0b20210209 Requires-Python >=3.6, <3.8; 0.1.0b20210210 Requires-Python >=3.6, <3.8; 0.1.0b20210211 Requires-Python >=3.6, <3.8; 0.1.0b20210212 Requires-Python >=3.6, <3.8; 0.1.0b20210213 Requires-Python >=3.6, <3.8; 0.1.0b20210214 Requires-Python >=3.6, <3.8; 0.1.0b20210215 Requires-Python >=3.6, <3.8; 0.1.0b20210216 Requires-Python >=3.6, <3.8; 0.1.0b20210217 Requires-Python >=3.6, <3.8; 0.1.0b20210218 Requires-Python >=3.6, <3.8; 0.1.0b20210219 Requires-Python >=3.6, <3.8; 0.1.0b20210220 Requires-Python >=3.6, <3.8; 0.1.0b20210221 Requires-Python >=3.6, <3.8; 0.1.0b20210222 Requires-Python >=3.6, <3.8; 0.1.0b20210223 Requires-Python >=3.6, <3.8; 0.1.0b20210224 Requires-Python >=3.6, <3.8; 0.1.0b20210225 Requires-Python >=3.6, <3.9; 0.1.0b20210226 Requires-Python >=3.6, <3.9; 0.1.0b

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rm_outlier(df):
    y_p75 = np.percentile(df[rate], 75)
    y_p25 = np.percentile(df[rate], 25)
    upper_bound = (y_p75-y_p25)*1.5+y_p75
    return df[df[rate] <= upper_bound]

In [ ]:
def train_test_split_df(df, test_size=0.2):
    test_df = df.sample(frac = test_size)
    train_df = df.drop(test_df.index)
    return train_df, test_df

In [ ]:
train_data = pd.read_csv('data/distance_from_airport_train_good.csv').drop(['tumbol_name','amphur_name','province_name'], axis=1)
rate = 'poverty_rate_living'
pov_rate = [c for c in train_data.columns if 'poverty_rate' in c and c != rate]
train_data = train_data.drop(pov_rate, axis=1)
# train_data['LAT'] = np.cos(train_data['LAT'])
# train_data['LONG'] = np.cos(train_data['LONG'])
train_data

In [ ]:
train_df, test_df = train_test_split_df(train_data)
print(train_df.shape, test_df.shape)
train_df = rm_outlier(train_df)
print(train_df.shape, test_df.shape)

In [ ]:
X_train = train_df.iloc[:, 1:-1].values
y_train = train_df.iloc[:, -1].values
print(X_train.shape, y_train.shape)

X_test = test_df.iloc[:, 1:-1].values
y_test = test_df.iloc[:, -1].values
print(X_test.shape, y_test.shape)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

# X_train_df
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklego.meta import ZeroInflatedRegressor

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=100)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

In [ ]:
zir = ZeroInflatedRegressor(
    classifier=ExtraTreesClassifier(),
    regressor=HuberRegressor(max_iter=1000)
)

zir.fit(X_train, y_train)

In [ ]:
y_pred = zir.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
mae

In [ ]:
mean_absolute_error(y_test, [0]*910)

In [ ]:
test_data = pd.read_csv('data/merged-data/test_v9.csv').drop('Region', axis=1)
test_data

In [ ]:
test_rm_nan = test_data.dropna()
test_rm_nan

In [ ]:
X_submit = test_rm_nan.iloc[:, 1:].values
# X_submit = pca.transform(X_submit)

In [ ]:
y_submit = zir.predict(X_submit)
y_submit = y_scaler.inverse_transform(y_submit.reshape(-1, 1))

In [ ]:
test_rm_nan[rate] = y_submit
submit_df = test_data.merge(test_rm_nan, on= 'tumbol_ID', how= 'left')[['tumbol_ID', 'poverty_rate_living']].fillna(0)
submit_df